In [2]:
from transformers import AutoTokenizer
import torch

In [16]:
token_budget = 25
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mathstral-7B-v0.1", padding_side="right")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
full_cots = ["A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?", "A robe takes 2 bolts of blue fiber and half that much white fiber."]
full_cots_tokens = tokenizer(full_cots, return_tensors="pt", padding=True, ).input_ids
print(full_cots_tokens)
cot_lengths = (full_cots_tokens != tokenizer.pad_token_id).sum(dim=1)
print(cot_lengths)
truncate_lengths = torch.minimum(cot_lengths, torch.tensor(token_budget))
print(truncate_lengths)
mask = torch.arange(full_cots_tokens.shape[1])[None, :] < truncate_lengths[:, None]
print(mask)
# Apply mask to get truncated sequences
part_cots_tokens = full_cots_tokens.masked_fill(~mask, tokenizer.pad_token_id)
part_cots = tokenizer.batch_decode(part_cots_tokens, skip_special_tokens=True)
part_cots

tensor([[    1, 29509,  1480,  1873,  5115, 29473, 29518, 17129,  1842,  1070,
          5813, 19635,  1072,  3563,  1137,  1956,  3843, 19635, 29491, 29473,
          2370,  2055, 17129,  1842,  1065,  3870,  2003,  1146,  2156, 29572],
        [    1, 29509,  1480,  1873,  5115, 29473, 29518, 17129,  1842,  1070,
          5813, 19635,  1072,  3563,  1137,  1956,  3843, 19635, 29491,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2]])
tensor([30, 19])
tensor([25, 19])
tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True, False,
         False, False, False, False, False, False, False, False, False, False]])


['A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in',
 'A robe takes 2 bolts of blue fiber and half that much white fiber.']